In [1]:
#================================================================================
# 신규 sentencepiece tokenizer vocab 생성 예제
# => tokenizer 방식중, SetnecePieceBPETokenzer 를 이용하여 새롭게 tokenize vocab 을 생성하는 예제임(GPT에서는 SetnecePieceBPETokenzer 를 이용함)
# => [과정 #2] 인 경우에는 에러 발생함. 따라서 [과정 #1]를 이용함
# => 참고로 [과정 #1] 은 kogpt-2 방식, [과정 #2]는 gpt-2 방식임.
# 
# 참고자료 
# https://gist.github.com/lovit/e11c57877aae4286ade4c203d6c26a32
# https://discuss.huggingface.co/t/training-sentencepiece-from-scratch/3477/2
#
#================================================================================
# [신규 생성 과정 #1] => Kogpt-2 tokenizer 방식
#
# 1. SetnecePieceBPETokenzer 정의 후 훈련 
#     => 훈련시, corpora 목록, vocab_size, min_frequency 등을 설정함
#
# 2. 훈련한 SetnecePieceBPETokenzer 를 PreTrainedTokenizerFast 와 연동
#     =>PreTrainedTokenizerFast(tokenizer_object=stokenizer)
#
# 3. PreTrainedTokenizerFast tokenizer 저장
#     => transforer_tokenizer.save_pretrained(OUT_PATH) 
#     => 정상적으로 저장되면, 해당 폴더에 3개 json 파일이 생성됨(tokenizer.json, special_tokens_map.json, tokenizer_config.json)
#================================================================================
# [신규 생성 과정 #2] => gpt-2 tokenizer 방식
#
# 1. SetnecePieceBPETokenzer 정의 후 훈련 
#     => 훈련시, corpora 목록, vocab_size, min_frequency 등을 설정함
#
# 2. 훈련한 SetnecePieceBPETokenzer를 저장
#     => stokenizer.save_model(OUT_PATH)
#     => 이대 저장되면, 해당 폴더에 2개의 파일이 생성됨(merges.txt, vocab.json)
#
# 3. GPT2TokenizerFast로 merges.txt, vocab.json 파일 불러옴
#     => transforer_tokenizer = GPT2TokenizerFast(vocab_file=vocab_path, merges_file=merges_path, add_prefix_space = True)
#
# 4. GPT2TokenizerFast tokenizer 저장
#     => transforer_tokenizer.save_pretrained(OUT_PATH) 
#     => 정상적으로 저장되면, 해당 폴더에 3개 json 파일이 생성됨(tokenizer.json, special_tokens_map.json, tokenizer_config.json)
# 
# *** 신규 생성과정 #2 테스트시, 3.GPT2TokenizerFast로 merges.txt, vocab.json 파일 불러와서, encode 테스트시 에러발생함(*원인모름)
#================================================================================

import torch
import numpy as np
import pandas as pd
import tokenizers
from tokenizers import (ByteLevelBPETokenizer,
                        CharBPETokenizer,
                        SentencePieceBPETokenizer,
                        BertWordPieceTokenizer)
# 말뭉치 지정
corpus_path = '../../korpora/kowiki_20190620/wiki_20190620_small.txt'


In [2]:
# 1. SetnecePieceBPETokenzer 정의 후 훈련 
stokenizer = SentencePieceBPETokenizer(add_prefix_space=True)

# 훈련
stokenizer.train(
    files = [corpus_path],
    vocab_size = 52000,  # 최대 vocab 계수 
    special_tokens = ["<cls>", "<eos>", "<mask>", "<unk>", "<pad>"],  # speical token 지정
    min_frequency = 5,   # 빈도수 
    show_progress = True,
    #limit_alphabet=10000, 
)

In [3]:
vocab = stokenizer.get_vocab()
print(f'vcoab 길이:{len(vocab)}')
print(sorted(vocab, key=lambda x: vocab[x]))  # sort 해서 vocab 출력 

vcoab 길이:10661
['<cls>', '<eos>', '<mask>', '<unk>', '<pad>', '\n', '\r', '!', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~', '«', '°', '·', '»', '×', 'Δ', 'α', 'μ', 'π', 'ᆞ', '–', '‘', '’', '“', '”', '⁄', '▁', '〈', '〉', '《', '》', '「', '」', '『', '』', '・', '가', '각', '간', '갈', '감', '갑', '값', '갔', '강', '갖', '같', '개', '객', '갬', '갱', '거', '건', '걸', '검', '것', '게', '겐', '겠', '겨', '격', '겪', '견', '결', '겸', '겼', '경', '계', '고', '곡', '곤', '곧', '골', '곱', '곳', '공', '과', '곽', '관', '괄', '광', '괴', '괵', '교', '구', '국', '군', '굳', '굴', '궁', '권', '궤', '귀', '규', '균', '그', '극', '근', '글', '금', '급', '긋', '긍', '기', '긴', '길', '김', '깊', 

In [4]:
# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = stokenizer.encode(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%stokenizer.decode(output.ids))

=>idx   : [9571, 2448, 1050, 6670, 5255, 647, 1554]
=>tokens: ['▁나는', '▁오늘', '▁아', '침을', '▁먹', '었', '다.']
=>offset: [(0, 2), (2, 5), (5, 7), (7, 9), (10, 12), (12, 13), (13, 15)]
=>decode: 나는 오늘 아침을 먹었다.



In [5]:
# 2. 훈련한 SetnecePieceBPETokenzer 를 PreTrainedTokenizerFast 와 연동
from transformers import PreTrainedTokenizerFast
transforer_tokenizer = PreTrainedTokenizerFast(tokenizer_object=stokenizer)

In [6]:
# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = transforer_tokenizer.encode(sentence)
print(output)
decode = transforer_tokenizer.decode(output)
print(decode)

[9571, 2448, 1050, 6670, 5255, 647, 1554]
나는 오늘 아침을 먹었다.


In [7]:
# PreTrainedTokenizerFast tokenizer 저장
import os
OUT_PATH = './mytoken'
os.makedirs(OUT_PATH, exist_ok=True)
transforer_tokenizer.save_pretrained(OUT_PATH)

('./mytoken/tokenizer_config.json',
 './mytoken/special_tokens_map.json',
 './mytoken/tokenizer.json')

In [9]:
from transformers import GPT2TokenizerFast

# 저장된 tokenzier 불러와서 확인
tokenizer = GPT2TokenizerFast.from_pretrained(OUT_PATH,
                                            bos_token='<cls>',
                                            eos_token='<eos>',
                                            unk_token='<unk>',
                                            pad_token='<pad>',
                                            mask_token='<mask>')

# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = transforer_tokenizer.encode(sentence)
print(output)
decode = transforer_tokenizer.decode(output)
print(decode)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'GPT2TokenizerFast'.


[9571, 2448, 1050, 6670, 5255, 647, 1554]
나는 오늘 아침을 먹었다.
